# Load microorganism

In [1]:
# Add path to the model
import sys
sys.path.append("../")
import cobra
from microorganism import Microorganism

# Read E. coli model from cobrapy
ecoli = cobra.io.load_json_model("../data/original/e_coli_core.json")
# Create a microorganism object
cobra_model = Microorganism(cobra_model=ecoli)
print(cobra_model)

e_coli_core (72 metabolites, 95 reactions)


### Create Metabolic Network

In [2]:
# add folder for imports
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm
from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data
from pathway import Pathway
from utils import create_compound, create_reaction
# suppres rdkit warnings
import rdkit
from rdkit import RDLogger
from utils import check_list

RDLogger.DisableLog('rdApp.*')


# read data from csv
cpds = pd.read_csv('../data/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('../data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('../data/pairs_final_RPAIRS_pred.csv', index_col=0)
cofactors = pd.read_csv('../data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)
pathway = Pathway()

data = create_compound(data, cpds, cofactors)
data = create_reaction(data, rxns)

# CREATE GRAPH
graph.create_graph(data=data, pairs=pairs)

graph.calculate_edge_mol_weight(data)
graph.calculate_smiles_similarity(data)

# nodes: 7997 
# edges: 11783


100%|██████████| 11783/11783 [01:25<00:00, 137.96it/s]


#### Get KEGG source and target

In [3]:
# define target compound
butanol_KEGG = 'C06142'
# define example source compound
oxoglatarate_KEGG = 'C00026'
# define source list
src_list = ['C00084', 'C00024', 'C00074', 'C00022', 'C00497', 'C00149', \
            'C00122', 'C00042', 'C00036', 'C00091', 'C00111', 'C00158']

#### Find metabolic pathways from src to trg

Find metabolic pathways from `Acetyl-coA` to `n-Butanol`

In [4]:
pathway.initialize(src_list[1], butanol_KEGG, graph)
pathway.get_pathway()
# pathway.print_pathway(with_reactions=True)

In [5]:
pathway.single_pathway(pathway.pred_paths[pathway.idx_smi[0]])

(['C00024', 'C00332', 'C01144', 'C00877', 'C00136', 'C01412', 'C06142'],
 [['R00238', 'R09527'],
  ['R01975', 'R01976'],
  ['R03026'],
  ['R01171', 'R01175', 'R09738', 'R11675'],
  ['R01172', 'R01173'],
  ['R03544', 'R03545']])

In [7]:
pathway.select_reactions(data, cobra_model)

for r in pathway.selected_pathway_reactions:
    print(r, ':', data.reactions[r].equation)

R00238 : 2 C00024 <=> C00010 + C00332
R01975 : C01144 + C00003 <=> C00332 + C00004 + C00080
R03026 : C01144 <=> C00877 + C00001
R01171 : C00136 + C00003 <=> C00877 + C00004 + C00080
R01172 : C01412 + C00010 + C00003 <=> C00136 + C00004 + C00080
R03544 : C01412 + C00004 + C00080 <=> C06142 + C00003


In [8]:
eqs_metabolites = pathway.reactions_add_gem(data, cobra_model, 'metabolites', save=True)

In [9]:
eqs_seed = pathway.reactions_add_gem(data, cobra_model, 'seed', save=True)

#### Extract reactions to add for all src

In [11]:
for s in src_list:
    print(s)
    pathway.initialize(s, butanol_KEGG, graph)
    pathway.get_pathway()
    pathway.single_pathway(pathway.pred_paths[pathway.idx_smi[0]])
    try:
        pathway.select_reactions(data, cobra_model)
    except ValueError:
        print('No pathway found')
        continue
    eqs_metabolites = pathway.reactions_add_gem(data, cobra_model, 'metabolites', save=True)
    eqs_seed = pathway.reactions_add_gem(data, cobra_model, 'seed', save=True)

C00084
C00024
C00074
C00022
C00497
No pathway found
C00149
C00122
C00042
No pathway found
C00036
No pathway found
C00091
No pathway found
C00111
C00158
